In [21]:
from db_client import DBClient
from analysis.data_clean import DataClean
import pandas as pd
import ast
from nltk.util import ngrams

In [3]:
def get_word_counts(bow_corpus):
    counts = {}
    for bow in bow_corpus:
        for word in bow:
            if word[0] not in counts.keys():
                counts[word[0]] = 0
            counts[word[0]] += word[1]
    return [(k, counts[k]) for k in sorted(counts, key=counts.get, reverse=True)]

def print_word_counts(word_counts, num_words, word_dict):
    for tup in word_counts[0:num_words]:
        print(f"{word_dict[tup[0]]}, {tup[1]} times")

In [4]:
# create db client and set seed
db = DBClient()
db.write(["SELECT setseed(0.5)"])

Connected to political tweets DB
db write committed


In [12]:
select_hash_dem = ["#bluewave2018", "#bluewave18", "#bluewave", "#democrats", "#resist", "#resistance", "#voteblue", 
"#Votethemout", "#WinBlue", "#YesWeCan", "#FlipItBlue"]

select_hash_rep = ["#trump", "#maga", "#gop", "#republican", "#trumptrain", "#kag", "#LeadRight", "#VoteRed", 
"#RedWave", "#VoteDemsOut"]

select_hash = select_hash_dem + select_hash_rep
dc = DataClean(select_hash)

In [5]:
data_query = """
with random_tweets as (
    select tweet_text_clean, Random() from staging.{}
    where tweet_date between '2018-01-01' and '2019-01-01'
    order by Random()
    limit 50000)
select tweet_text_clean 
from random_tweets;
"""

In [19]:
docs = []
for table in ["democrat", "republican", "house", "senate"]:
    tweets = pd.DataFrame(db.read(data_query.format(table)))
    docs = docs + [ast.literal_eval(doc) for doc in tweets[0].tolist()]
    

['@knoone413',
 '@superstimpy',
 '@nbcolympics',
 'insult',
 '#socialism',
 '#feelthebern',
 'folk',
 'know',
 'ask',
 'also',
 '#imwithher',
 '#democrats',
 'also',
 'pull',
 '#fakenews',
 'especially',
 '@cnn',
 '@nytimes',
 '#maga',
 '#trumptrain']

In [20]:
def bigrams(tweet):
    return ["_".join(w) for w in ngrams(tweet, 2)]

In [25]:
bigrams = list(map(bigrams, docs))

/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [32]:
bigrams =[tweet for tweet in bigrams if len(tweet) > 0]

In [33]:
len(bigrams)

192580